In [3]:
# creating a vector redis store
from config import embeddings, REDIS_URL, r
from langchain_redis import RedisConfig, RedisVectorStore

config = RedisConfig(
    index_name="documents",
    redis_url=REDIS_URL,
    metadata_schema=[
        {"name": "category", "type": "tag"},
    ],
)

vector_store = RedisVectorStore(embeddings, config=config)

21:21:45 redisvl.index.index INFO   Index already exists, not overwriting.


In [ ]:
import json
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# handling JSON files
def json_adder(filename, vector_store):

    # each JSON file has to have a category and a content

    # Loading the JSON file
    with open(filename, 'r') as f:
        json_data = json.load(f)

    # Extract texts and metadata from the JSON file
    json_texts = [item["content"] for item in json_data]
    json_metadata = [{"category": item["category"]} for item in json_data]

    # Add JSON data to Redis vector store
    vector_store.add_texts(json_texts, json_metadata)

# handling PDF files
def pdf_adder(filename, vector_store):

    # Load the PDF
    pdf_loader = PyPDFLoader(filename)
    pdf_docs = pdf_loader.load()

    # Split the PDF into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    pdf_splits = text_splitter.split_documents(pdf_docs)

    # Extract texts and metadata from PDF splits
    pdf_texts = [chunk.page_content for chunk in pdf_splits]
    pdf_metadata = [{"page_number": chunk.metadata.get("page", "unknown"), "category": "pdf"} for chunk in pdf_splits]

    vector_store.add_texts(pdf_texts, pdf_metadata)

# Adding JSON files
json_location = "/home/krispy_noodles/SIT_Chatbot/assets/json/SIT_General_FAQ.json"
# json_adder(json_location, vector_store)

# # Add PDF chunks to Redis vector store
pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/Rules-and-Regulations-of-SIT-Learner-Behaviour-Final.pdf"
# pdf_adder(pdf_location, vector_store)

In [ ]:
# from redis_db import new_vector_store

# Adding JSON files
json_location = "/home/krispy_noodles/SIT_Chatbot/assets/json/CCR_data.json"
json_adder(json_location, vector_store)

json_location = "/home/krispy_noodles/SIT_Chatbot/assets/json/course_links.json"
json_adder(json_location, vector_store)

json_location = "/home/krispy_noodles/SIT_Chatbot/assets/json/Graduate_Employment_Survey_2023.json"
json_adder(json_location, vector_store)

json_location = "/home/krispy_noodles/SIT_Chatbot/assets/json/sit_data.json"
json_adder(json_location, vector_store)

# Add PDF chunks to Redis vector store
pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/matriculation_guide.pdf"
pdf_adder(pdf_location, vector_store)

pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/Rules-and-Regulations-of-SIT-Learner-Behaviour-Final.pdf"
pdf_adder(pdf_location, vector_store)

pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/SIT_Annual_Report.pdf"
pdf_adder(pdf_location, vector_store)

pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/SIT_handbook.pdf"
pdf_adder(pdf_location, vector_store)

pdf_location = "/home/krispy_noodles/SIT_Chatbot/assets/pdf/Web_Publication_SIT GES_2023.pdf"
pdf_adder(pdf_location, vector_store)

Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 830 0 (offset 0)


In [16]:
import redis
from dotenv import dotenv_values

# retrieving the env file
env_vars = dotenv_values(".env")

REDIS_URL = env_vars["REDIS_URL"]

r = redis.Redis(
  host='redis-19084.c10.us-east-1-4.ec2.redns.redis-cloud.com',
  port=19084,
  password=env_vars["REDIS_PW"]
)

# converting the vector store into a yaml file
yml_file_name = "redis_schema.yaml"

vector_store.index.schema.to_yaml(yml_file_name)

# Read the YAML file contents
with open(yml_file_name, 'r') as f:
    schema_yaml = f.read()

# Store the YAML schema in Redis
r.set("vector_store_schema", schema_yaml)

response = r.ping()
print(response)

True


In [15]:
from redis_db import vector_store

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})
reply = retriever.invoke("Why join SIT?")

reply

21:26:24 redisvl.index.index INFO   Index already exists, not overwriting.


[Document(metadata={'category': 'General FAQ'}, page_content="Question: What is SIT's Mission? Answer: To maximise the potential of our learners and to innovate with industry, through an integrated applied learning and research approach, so as to contribute to the economy and society."),
 Document(metadata={'category': 'General FAQ'}, page_content="Question: What is unique about SIT? Answer: As Singapore's first university of applied learning, we develop individuals and innovate with industry to impact the economy and society in meaningful ways. And we do this through our unique applied learning pedagogy. Read more about our latest successes and happenings on our [newsroom](https://www.singaporetech.edu.sg/news) and learn more about the SIT experience on the [Why SIT page](https://www.singaporetech.edu.sg/why-sit)."),
 Document(metadata={'category': 'pdf'}, page_content='SIT for one trimester, while concurrently pursuing \ntheir degree programme in their home university. \nLikewise, SI

In [ ]:
# clearing redis database

# from config import r

# # Delete all keys in db
# r.flushdb()

True